In [1]:
import sys
import os
from PIL import Image
from matplotlib.pyplot import imshow
import numpy as np
import io
import cv2
import re

In [2]:
'''
- expected folder structure: the source frames should be arranged as follows:

Montalbano:
 - training:
   - Samplexxxxx_color
       - 0.jpg
       - 1.jpg
       - 2.jpg
       - ...
   - Samplexxxxx_color
 - test:
   - Samplexxxxx_color
   - Samplexxxxx_color
 - validation:
   - Samplexxxxx_color
   - Samplexxxxx_color
   
GRIT:
 - abort:
   - 1_0
       - 2015-03-30 15_21_36.965829.jpg
       - 2015-03-30 15_21_37.045439.jpg
       - 2015-03-30 15_21_37.127836.jpg
       - ...
   - 1_1
 - circle:
   - 1_1
   - 1_2
 - ...


'''

'\n- expected folder structure: the source frames should be arranged as follows:\n\nMontalbano:\n - training:\n   - Samplexxxxx_color\n       - 0.jpg\n       - 1.jpg\n       - 2.jpg\n       - ...\n   - Samplexxxxx_color\n - test:\n   - Samplexxxxx_color\n   - Samplexxxxx_color\n - validation:\n   - Samplexxxxx_color\n   - Samplexxxxx_color\n   \nGRIT:\n - abort:\n   - 1_0\n       - 2015-03-30 15_21_36.965829.jpg\n       - 2015-03-30 15_21_37.045439.jpg\n       - 2015-03-30 15_21_37.127836.jpg\n       - ...\n   - 1_1\n - circle:\n   - 1_1\n   - 1_2\n - ...\n\n\n'

In [3]:
def differential_images(frames, filenames):
    result = []
    for index, frame in enumerate(frames[1:-1]):
        temp = cv2.bitwise_and(cv2.absdiff(frames[index + 2], frames[index + 1]), cv2.absdiff(frames[index + 1], frames[index]))
        #imshow(temp)
        result.append(temp)
    return result, filenames

In [4]:
def read_frames(path, timestamp = False):
    frames = []
    filenames = []
    if timestamp:
        sorted_dir = sorted(os.listdir(path))
    else:
        sorted_dir = sorted(os.listdir(path), key=lambda x: int(re.split(r'(\d+)', x)[1])) #sort by name, consider -start, -end
    for filename in sorted_dir:
        frames.append(cv2.resize(cv2.imread(os.path.join(path,filename), 0), (64, 48))) #read as grayscale, resize
        #frames.append(cv2.resize(cv2.imread(os.path.join(path,filename), 0), (640, 480))) #use for testing
        filenames.append(filename)
    return frames, filenames #frames, names (needed for writing)

In [5]:
def process(path, target_path, timestamp = False):
    for folder in os.listdir(path): #training, test, etc. circle, abort #etc
        target_folder = os.path.join(target_path, folder)
        if not os.path.exists(target_folder):
            os.mkdir(target_folder)
        for subfolder in os.listdir(os.path.join(path, folder)): #Sample00021_color, Sample00022_color, etc. 1_0, 1_1 etc.
            if (subfolder.startswith(str(""))): #use for testing
                print(subfolder)
                frames, filenames = read_frames(os.path.join(path, folder, subfolder), timestamp)
                frames, filenames = differential_images(frames, filenames)
                #print(filenames)
                count = 1
                for index, frame in enumerate(frames):
                    target_subfolder = os.path.join(target_path, folder, subfolder)
                    if not os.path.exists(target_subfolder):
                        os.mkdir(target_subfolder)
                    if timestamp:
                        cv2.imwrite(os.path.join(target_path, folder, subfolder, str(count)+'.jpg'), frame)
                    else:
                        cv2.imwrite(os.path.join(target_path, folder, subfolder, str(filenames[index])), frame)
                    count = count + 1


In [6]:
# Montalbano
path = '/path/to/montalbano_frame_source' #path to dataset (isolated sequences)
target_path = '/path/to/montalbano_preprocessed_frames' #any existing folder
process(path, target_path)
# GRIT
path = '/path/to/grit_frame_source' #path to dataset (isolated sequences)
target_path = '/path/to/grit_preprocessed_frames' #any existing folder
process(path, target_path, True)

Sample00308_color
Sample00309_color
Sample00310_color
Sample00629_color
Sample00631_color
Sample00630_color
Sample01054_color
Sample01056_color
Sample01055_color
